In [4]:
import zipfile
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('all')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report, recall_score
import warnings
from pandas.errors import SettingWithCopyWarning
import joblib
import tensorflow as tf
import os
import torch
from transformers import BertTokenizer, BertModel
import umap
import hdbscan
from sklearn.linear_model import LogisticRegression
from bertopic import BERTopic

uploaded_zip = '/home/abarovic/popeREU/eclipse_platform.zip'
extract_dir = '/home/abarovic/popeREU'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(uploaded_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

csv_file = os.path.join(extract_dir, 'eclipse_platform.csv')
fulldata = pd.read_csv(csv_file)

print(fulldata.columns)
print(fulldata.isna().sum())  # Check NaNs in training features

def CreateDuration(dataset):
    dataset['Created_time'] = pd.to_datetime(dataset['Created_time'], utc=True)
    dataset['Resolved_time'] = pd.to_datetime(dataset['Resolved_time'], utc=True)

    # Convert to the desired format
    dataset['Created_time2'] = dataset['Created_time'].dt.strftime('%m/%d/%Y %H:%M')
    dataset['Resolved_time2'] = dataset['Resolved_time'].dt.strftime('%m/%d/%Y %H:%M')

    # Parse the formatted datetime strings back to datetime objects to ensure they are in the correct format
    dataset['Created_time2'] = pd.to_datetime(dataset['Created_time2'], format='%m/%d/%Y %H:%M')
    dataset['Resolved_time2'] = pd.to_datetime(dataset['Resolved_time2'], format='%m/%d/%Y %H:%M')

    # Calculate the duration in hours
    dataset['Duration'] = dataset['Resolved_time2'] - dataset['Created_time2']
    dataset['Duration_hours'] = dataset['Duration'].dt.total_seconds() / 3600

    # Drop the intermediate columns
    dataset.drop(['Created_time2', 'Resolved_time2', 'Duration'], axis=1, inplace=True)

CreateDuration(fulldata)

def NoOutliers(df):
    Q1 = df['Duration_hours'].quantile(0.25)
    Q3 = df['Duration_hours'].quantile(0.75)

    # Calculate the IQR
    IQR = Q3 - Q1

    # Define the lower and upper bounds for filtering outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out the outliers
    df = df[(df['Duration_hours'] >= lower_bound) & (df['Duration_hours'] <= upper_bound)]
    return(df)

#fulldata = NoOutliers(fulldata)

def CreateTimeLabel(df):
    threshold = df['Duration_hours'].quantile(0.70)
    # Create a new column TimeLabel based on the threshold
    df['TimeLabel'] = df['Duration_hours'].apply(lambda x: 'short' if x <= threshold else 'long')
    df = df[df['Resolution'] != 'MOVED']
    #df = df[df['Resolution'] == 'FIXED']
    return df

fulldata = CreateTimeLabel(fulldata)
def RemoveStopWords(dataset):
    #making all the items in the descriptions columns lower
    dataset['Description'] = dataset['Description'].str.lower()
    # Define stop words
    stop_words = set(stopwords.words('english'))
     # Remove stop words
    for index, row in dataset.iterrows():
        # Check if the 'Description' is not NaN
        if isinstance(row['Description'], str):
            words = row['Description'].split()
            final_tokens = [word for word in words if word not in stop_words]
            dataset.at[index, 'Description'] = ' '.join(final_tokens)
    return dataset

fulldata = RemoveStopWords(fulldata)

def Lemmitization(dataset):
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Function to lemmatize a sentence
    def lemmatize_sentence(sentence):
        words = nltk.word_tokenize(sentence)
        lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Lemmatize verbs (default)
        return ' '.join(lemmatized_words)
    dataset['Description'] = dataset['Description'].astype(str)  # Convert all to strings
    # Apply lemmatization to each row in the 'Description' column
    dataset['Description'] = dataset['Description'].apply(lemmatize_sentence)

Lemmitization(fulldata)

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None

# Function to calculate sentiment scores
def CalculateSentimentScores(description):
    tokens = word_tokenize(description)
    tagged_tokens = pos_tag(tokens)
        
    pos_score = 0
    neg_score = 0
    token_count = 0
    for word, tag in tagged_tokens:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
            continue
        
        synsets = list(swn.senti_synsets(word, wn_tag))
        if not synsets:
            continue
        
        # Use the first synset for simplicity
        synset = synsets[0]
        pos_score += synset.pos_score()
        neg_score += synset.neg_score()
        token_count += 1
    
    # Normalize scores by the number of tokens
    if token_count > 0:
        pos_score /= token_count
        neg_score /= token_count
        return pos_score, neg_score

# Apply sentiment score calculation to each description
def CreatePosNegColumns(dataset): 
    dataset[['Pos_Score', 'Neg_Score']] = dataset['Description'].apply(lambda x: pd.Series(CalculateSentimentScores(x)))

CreatePosNegColumns(fulldata)

def EmotionColumn(dataset):
    #Creating a column for binary emotion (positive or negative)
    dataset['Emotion'] = dataset['Pos_Score'] - dataset['Neg_Score']
    
    # Assign labels based on the difference
    dataset['Emotion'] = dataset['Emotion'].apply(lambda x: 'positive' if x > 0 else 'negative')

EmotionColumn(fulldata)

def EmotionalityColumn(dataset):
    dataset['Emotionality'] = dataset['Pos_Score']+ dataset['Neg_Score']

EmotionalityColumn(fulldata)

def Destiny(df):
    # Create a new column 'Destiny' with default value 'Not Fixed'
    df['Destiny'] = 'Not Fixed'
    # Update 'Destiny' based on conditions in 'Resolution' column
    df.loc[df['Resolution'] == 'FIXED', 'Destiny'] = 'Fixed'
    return df

fulldata = Destiny(fulldata)
#print(fulldata['Emotion'].head(100))

def Emotion(df):
    # Fill NaN values with a default value (e.g., 'neutral') before mapping
    df['Emotion'] = df['Emotion'].fillna('neutral')
    # Label Positive emotion as 1, negative as 0, and neutral as -1
    df['Emotion'] = df['Emotion'].map({'positive': 1, 'negative': 0, 'neutral': -1})
    return df

fulldata = Emotion(fulldata)


def Priority(df):
    # Fill NaN values with a default value (e.g., 'P5') before mapping
    df['Priority'] = df['Priority'].fillna('P5')
    # Label Priority Columns numerically
    df['Priority'] = df['Priority'].map({'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5})
    return df

fulldata = Priority(fulldata)

def DropDescriptionNA(df):
    df = df[df['Description'] != 'nan']

DropDescriptionNA(fulldata)

def Model(df):
    model = BERTopic(nr_topics=20)

    # Prepare the documents
    docs = df['Description'].fillna('').astype(str).tolist()

    # Fit the model to your data
    topics, probabilities = model.fit_transform(docs)
    #adding a column for predicted topics
    new_docs = df['Description'].fillna('').astype(str).to_list()
    topics, probs = model.transform(new_docs)
    df['Predicted_Topic'] = topics

    # Get topic information
    topic_info = model.get_topic_info()
    print(topic_info)
    # Get words associated with each topic
    topic_words = {topic: model.get_topic(topic) for topic in topic_info.Topic}
    print(topic_words)
    return df

fulldata = Model(fulldata)

print(fulldata.columns)

print(fulldata.columns)
print(fulldata['Emotionality'].head(100))

def SplitData(dataset):
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    trainset, testset = dataset[:train_size], dataset[train_size:]
    return trainset, testset
trainset, testset = SplitData(fulldata)
print("Total items in dataset:", fulldata.shape[0])
print("Total items in trainset:", trainset.shape[0])
print("Total items in testset:", testset.shape[0])

print(trainset.columns)
print(testset.columns)

print(trainset['Emotionality'].head(100))

print("-----------------\n Training Set: \n-----------------")
print(trainset.isna().sum())  # Check NaNs in training features
print("-----------------\n Test Set: \n-----------------")
print(testset.isna().sum())   # Check NaNs in test features


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]  

Index(['Issue_id', 'Priority', 'Component', 'Duplicated_issue', 'Title',
       'Description', 'Status', 'Resolution', 'Version', 'Created_time',
       'Resolved_time'],
      dtype='object')
Issue_id                0
Priority                0
Component               0
Duplicated_issue    70752
Title                   0
Description           129
Status                  0
Resolution              0
Version                 0
Created_time            0
Resolved_time           0
dtype: int64


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

    Topic  Count                                               Name  \
0      -1  40245                           -1_file_eclipse_line_new   
1       0  30189                         0_at_00000000_file_eclipse   
2       1   9524                           1_new_shell_text_display   
3       2    897                           2_patch_bug_apply_create   
4       3    834                        3_search_cheat_sheet_result   
5       4    798                           4_print_console_0303_gtk   
6       5    738                              5_note_w00t_dsgsg_den   
7       6    611                         6_bind_object_return_value   
8       7    536                    7_attachment_create_none_attach   
9       8    220                   8_sort_column_order_viewersorter   
10      9    137                   9_tool_external_builder_builders   
11     10    129                                          10_nan___   
12     11     76                              11_html_lt_xhtml_body   
13    

MLP (Emotion, Emotionality, priority)

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']

# Define features and target
features = ['Emotion', 'Emotionality', 'Priority']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Train MLP Classifier
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Predictions
y_pred = mlp.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.7937870427140813
              precision    recall  f1-score   support

           0       0.79      1.00      0.88      7748
           1       0.73      0.02      0.03      2038

    accuracy                           0.79      9786
   macro avg       0.76      0.51      0.46      9786
weighted avg       0.78      0.79      0.71      9786



MLP (Emotion, Emotionality, priority, BERT)

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']

# Define features and target
features = ['Emotion', 'Emotionality', 'Priority', 'Predicted_Topic']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Train MLP Classifier
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Predictions
y_pred = mlp.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7937870427140813
              precision    recall  f1-score   support

           0       0.79      1.00      0.88      7748
           1       0.73      0.02      0.03      2038

    accuracy                           0.79      9786
   macro avg       0.76      0.51      0.46      9786
weighted avg       0.78      0.79      0.71      9786



MLP (Emotion, Emotionality, priority) -> Weighted

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from collections import Counter

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']
# Define features and target
features = ['Emotion', 'Emotionality', 'Priority']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("Resampled class distribution:", Counter(y_train))

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train MLP Classifier
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Predictions
y_pred = mlp.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Resampled class distribution: Counter({1: 24490, 0: 24490})
Accuracy: 0.6367259350091968
              precision    recall  f1-score   support

           0       0.80      0.72      0.76      7748
           1       0.23      0.31      0.26      2038

    accuracy                           0.64      9786
   macro avg       0.51      0.51      0.51      9786
weighted avg       0.68      0.64      0.66      9786



MLP (Emotion, Emotionality, priority, BERT) -> Weighted

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from collections import Counter

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']

# Define features and target
features = ['Emotion', 'Emotionality', 'Priority', 'Predicted_Topic']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("Resampled class distribution:", Counter(y_train))

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train MLP Classifier
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Predictions
y_pred = mlp.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Resampled class distribution: Counter({1: 24490, 0: 24490})
Accuracy: 0.5793991416309013
              precision    recall  f1-score   support

           0       0.81      0.62      0.70      7748
           1       0.23      0.44      0.30      2038

    accuracy                           0.58      9786
   macro avg       0.52      0.53      0.50      9786
weighted avg       0.69      0.58      0.62      9786

